# Gamma Parameter Range 

Conclusion: Based on the results here, I propose a gamma range of:  
`gammas = [0.001, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0 200.0]`

In [2]:
from res_experiment import *
import time

In [18]:
def range_finder(par_range = np.logspace(-7,5, 13), ORBITS_PER = 10, parameter = "gamma", alt_topology = False):
    # Standard Hyper Parameters
    params = {
        "uniform_weights": True,
        "solver": "ridge",
        "ridge_alpha": .0001,
        "signal_dim": 3,
        "network": "random graph",
        "res_sz": 2000,
        "activ_f": np.tanh,
        "connect_p": .001,
        "spect_rad": .9,
        "gamma": 1.,
        "sigma": 0.12,
        "sparse_res": True,
    }

    timesteps_correct = []
    errors = []
    times = []
    
    for p in par_range:
        err = []
        acc = []
        start = time.time()
        for i in range(ORBITS_PER):
            # Solve lorenz equations for a random initial condition
            X0 = random_lorenz_x0()
            train_t, test_t, u = lorenz_equ(X0)
            # Make reservoir computer
            params[parameter] = p
            if alt_topology:
                #For testing on alternate topology could be expanded to let you choose topologies
                A = barab1()
                rc = ResComp(A, **params)
            else:
                rc = ResComp(**params)
            # Train reservoir computer and predict
            err.append(rc.fit(train_t, u))
            pred = rc.predict(test_t)
            acc.append(how_long_accurate(u(test_t), pred))
        # Store results
        print(f"Gamma: {p} \t Timesteps Correct: {np.mean(acc)} \t Error: {np.mean(err)} \t Time: {format(time.time()-start, '.2f')}")
        times.append(time.time()-start)
        errors.append(np.mean(err))
        timesteps_correct.append(np.mean(acc))
    return [par_range,times,errors,timesteps_correct]
        

In [9]:
test1 = range_finder()

Gamma: 1e-07 	 Timesteps Correct: 0.0 	 Error: 12.403528213166302 	 Time: 55.22
Gamma: 1e-06 	 Timesteps Correct: 0.0 	 Error: 12.247515314295494 	 Time: 59.84
Gamma: 1e-05 	 Timesteps Correct: 0.0 	 Error: 12.275941206325385 	 Time: 62.39
Gamma: 0.0001 	 Timesteps Correct: 0.0 	 Error: 8.523098038761303 	 Time: 65.70
Gamma: 0.001 	 Timesteps Correct: 4.6 	 Error: 2.8697629211327382 	 Time: 68.94
Gamma: 0.01 	 Timesteps Correct: 124.6 	 Error: 0.8379792293314804 	 Time: 74.00
Gamma: 0.1 	 Timesteps Correct: 279.7 	 Error: 0.3169261868102395 	 Time: 87.67
Gamma: 1.0 	 Timesteps Correct: 933.6 	 Error: 0.06640700293343804 	 Time: 93.47
Gamma: 10.0 	 Timesteps Correct: 736.9 	 Error: 0.016924013296318437 	 Time: 124.28
Gamma: 100.0 	 Timesteps Correct: 540.7 	 Error: 0.007845110497030846 	 Time: 373.47
Gamma: 1000.0 	 Timesteps Correct: 173.8 	 Error: 0.005290153289184412 	 Time: 1371.81


KeyboardInterrupt: 

In [11]:
from matplotlib import pyplot as plt
def range_grapher(test,parameter="gamma"):
    plt.plot(test[0],test[1])
    plt.title("Parameters vs time")
    plt.xlabel(parameter)
    plt.ylabel("time")
    plt.show()
    plt.plot(test[0],test[2])
    plt.title("Parameters vs error")
    plt.xlabel(parameter)
    plt.ylabel("error")
    plt.show()
    plt.plot(test[0],test[3])
    plt.title("Parameters vs timesteps")
    plt.xlabel(parameter)
    plt.ylabel("timesteps")
    plt.show()

NameError: name 'test1' is not defined

When gamma was equal to 1000, the fit was extremely slow—probably due to poorly conditioned matrixes

## Repeat with less regularization and a tighter range

In [16]:
gms = np.logspace(-3,2.5, 14)
test2 = range_finder(gms)
range_grapher(test2)

Gamma: 0.001 	 Timesteps Correct: 2.3 	 Error: 2.859448456184267 	 Time: 67.38
Gamma: 0.0026489692876105267 	 Timesteps Correct: 32.6 	 Error: 1.6577813586610783 	 Time: 68.13
Gamma: 0.00701703828670383 	 Timesteps Correct: 33.8 	 Error: 1.0026275793552122 	 Time: 74.02
Gamma: 0.018587918911465634 	 Timesteps Correct: 77.8 	 Error: 0.6704868641585413 	 Time: 75.25
Gamma: 0.04923882631706739 	 Timesteps Correct: 364.3 	 Error: 0.45438969978396687 	 Time: 78.53
Gamma: 0.13043213867190054 	 Timesteps Correct: 288.2 	 Error: 0.25302725873379456 	 Time: 83.22
Gamma: 0.34551072945922184 	 Timesteps Correct: 499.5 	 Error: 0.12996647322069013 	 Time: 86.92
Gamma: 0.9152473108773893 	 Timesteps Correct: 1240.2 	 Error: 0.06284864572415358 	 Time: 88.01
Gamma: 2.424462017082328 	 Timesteps Correct: 1666.5 	 Error: 0.039762373812831965 	 Time: 95.18
Gamma: 6.422325422229356 	 Timesteps Correct: 1042.1 	 Error: 0.022357488300640826 	 Time: 111.07
Gamma: 17.012542798525892 	 Timesteps Correct: 108

NameError: name 'timesteps' is not defined

## Repeat with a different network topology

In [11]:
gms = np.logspace(-3,2.5, 14)
test3 = range_finder(gms,alt_topology=True)
range_grapher(test3)

Gamma: 0.001 	 Timesteps Correct: 104.0 	 Error: 5.8336803058467845
Gamma: 0.0026489692876105267 	 Timesteps Correct: 0.0 	 Error: 3.498813933215292
Gamma: 0.00701703828670383 	 Timesteps Correct: 0.0 	 Error: 2.4714473473403937
Gamma: 0.018587918911465634 	 Timesteps Correct: 110.0 	 Error: 1.334837981618017
Gamma: 0.04923882631706739 	 Timesteps Correct: 0.0 	 Error: 0.9056538197756402
Gamma: 0.13043213867190054 	 Timesteps Correct: 467.0 	 Error: 0.4598280028937956
Gamma: 0.34551072945922184 	 Timesteps Correct: 328.0 	 Error: 0.28142107489075774
Gamma: 0.9152473108773893 	 Timesteps Correct: 1295.0 	 Error: 0.11514407339990687
Gamma: 2.424462017082328 	 Timesteps Correct: 1166.0 	 Error: 0.0792655933353127
Gamma: 6.422325422229356 	 Timesteps Correct: 964.0 	 Error: 0.0347717632923852
Gamma: 17.012542798525892 	 Timesteps Correct: 302.0 	 Error: 0.018149027422052787


KeyboardInterrupt: 